___
___
## <font color='coral'>**Appel de l'environment et chargement des données** </font> 

In [1]:
%reset -f
import ipywidgets as widgets
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage.filters import uniform_filter1d, gaussian_filter1d
from IPython.display import clear_output

## For sliders
from ipywidgets import Layout, interact

## Set up rc
plt.rcParams['image.cmap'] = 'RdYlBu_r'
plt.rcParams['axes.formatter.limits'] = (-3,1)
plt.rcParams['lines.linewidth'] = 2.5
plt.rcParams.update({"axes.grid" : True})
plt.rcParams['lines.markersize'] = 8
plt.rcParams['lines.markeredgecolor'] = 'k'
plt.rcParams['lines.markeredgewidth'] = 2.0
plt.rc('font',size=18)
me = (0.1,0.1) #Markevery

import h5py

def hdf5_to_dict(filename):
    """Load a dictionary of arrays and strings as unicode characters from an HDF5 file."""
    with h5py.File(filename, 'r') as f:
        d = {}
        for k in f.keys():
            v = f[k][()]
            if isinstance(v, bytes):
                d[k] = v.decode('utf-8')
            else:
                d[k] = np.array(v)
    return d

___
___

<a id='exploration_radial_profile'></a>
## <font color='xkcd:sage'>**Exploration**</font> - Carte $(r,\theta)$ des sources

In [2]:
## Load data
q05_sources = hdf5_to_dict('data_2D/q05_sources_r020.h5')
q15_sources = hdf5_to_dict('data_2D/q15_sources_r020.h5')

## Plot 2D arrays in polar coordinates
data = [q05_sources, q15_sources]
def update_plot(keys):
    fig = plt.figure(figsize=(20,10))
    

    for iq,q in enumerate(data):
        ax = fig.add_subplot(1,len(data),iq+1, projection='polar')
        theta_ar  = np.linspace(0,2*np.pi,q[keys].shape[0])
        radial_ar = np.linspace(0,1.3,q[keys].shape[1])
        p = ax.pcolormesh(theta_ar, radial_ar, q[keys].T, cmap='RdYlBu_r')
        fig.colorbar(p, ax=ax)
    # Add a r=1 circle
    ax.plot(np.linspace(0,2*np.pi,100), np.ones(100), 'k--')

keys_widget = widgets.Select(options=list(q05_sources), value='Vorticity_Sce', layout=Layout(width='50%', height='220px'))

interact(update_plot, 
         keys = keys_widget
        )

interactive(children=(Select(description='keys', index=11, layout=Layout(height='220px', width='50%'), options…

<function __main__.update_plot(keys)>

<a id='exploration_radial_profile'></a>
## <font color='xkcd:sage'>**Exploration**</font> - Carte $(r,\theta)$ des fichiers rprof_GC2D à $t=31000 [\omega_{c0}^{-1}]$

In [3]:
## Load data
q05_rprof2D = hdf5_to_dict('data_2D/q05_rprof2D_t31000.h5'); q05_rprof2D['title'] = 'q05'
q15_rprof2D = hdf5_to_dict('data_2D/q15_rprof2D_t31000.h5'); q15_rprof2D['title'] = 'q15'

## Plot 2D arrays in polar coordinates
data = [q05_rprof2D, q15_rprof2D]
def update_plot(keys,rhomax,gradr_bool,coef_adjust):
    fig = plt.figure(figsize=(20,10))

    for iq,q in enumerate(data):
        ax = fig.add_subplot(1,len(data),iq+1, projection='polar')
        theta_ar  = np.linspace(0,2*np.pi,q[keys].shape[0])
        radial_ar = np.linspace(0,1.3,q[keys].shape[1])

        irhomax = np.argmin(np.abs(radial_ar-rhomax))

        

        if gradr_bool : 
            vmax = np.amax(np.abs(np.gradient(data[1][keys],radial_ar*250,axis=1)[:,:irhomax])) * coef_adjust
            vmin = -vmax
            p = ax.pcolormesh(theta_ar, radial_ar[:irhomax], np.gradient(q[keys],radial_ar*250,axis=1)[:,:irhomax].T, cmap='RdYlBu_r', vmin=vmin, vmax=vmax)
        else: 
            vmax = np.amax(np.abs(data[1][keys][:,:irhomax])) * coef_adjust
            vmin = -vmax
            p = ax.pcolormesh(theta_ar, radial_ar[:irhomax], q[keys][:,:irhomax].T, cmap='RdYlBu_r', vmin=vmin, vmax=vmax)
        fig.colorbar(p, ax=ax)

        ax.set_title(q['title'])
        # Add a r=1 circle
        ax.plot(np.linspace(0,2*np.pi,100), np.ones(100), 'k--')

keys_widget   = widgets.Select(options=list(data[0]), value=list(data[0])[0], layout=Layout(width='50%', height='220px'))
rhomax_widget = widgets.FloatSlider(value=1, min=0, max=1.3, step=0.05, description='Normalized radius range', layout=Layout(width='50%'), style = {'description_width': 'initial'}, continuous_update=False)
gradr_bool_widget = widgets.Checkbox(value=False, description='Gradr', disabled=False, indent=False)
coef_adjust_widget = widgets.FloatSlider(value=1, min=0.02, max=1, step=0.02, description='colormap_adjust', layout=Layout(width='50%'), style = {'description_width': 'initial'}, continuous_update=False)

interact(update_plot, 
         keys        = keys_widget,
         rhomax      = rhomax_widget,
         gradr_bool  = gradr_bool_widget,
         coef_adjust = coef_adjust_widget
        )

interactive(children=(Select(description='keys', layout=Layout(height='220px', width='50%'), options=('IturbGC…

<function __main__.update_plot(keys, rhomax, gradr_bool, coef_adjust)>

<a id='exploration_radial_profile'></a>
## <font color='xkcd:grey green'>**Test**</font> - Visualisation des oscillations radiale

In [4]:
## Load data
q05_rprof2D = hdf5_to_dict('data_2D/q05_rprof2D_t31000.h5'); q05_rprof2D['title'] = 'q05'
q15_rprof2D = hdf5_to_dict('data_2D/q15_rprof2D_t31000.h5'); q15_rprof2D['title'] = 'q15'

## Plot 2D arrays in polar coordinates
data = [q05_rprof2D, q15_rprof2D]
fig = plt.figure(figsize=(20,10))

keys = 'densGC_rtheta'

for iq,q in enumerate(data):
    ax = fig.add_subplot(len(data),1,iq+1)
    theta_ar  = np.linspace(0,2*np.pi,q[keys].shape[0])
    radial_ar = np.linspace(0,1.3,q[keys].shape[1])

    irhomin = np.argmin(np.abs(radial_ar-0.6))
    irhomax = np.argmin(np.abs(radial_ar-0.75))
    itheta_90deg = np.argmin(np.abs(theta_ar-np.pi/2))
        
    p = ax.plot(radial_ar[irhomin:irhomax], np.gradient(q[keys],radial_ar*250,axis=1)[itheta_90deg,irhomin:irhomax])

    for rho in radial_ar[irhomin:irhomax]:
        ax.axvline(rho, color='k', linestyle='--', alpha=0.2)

    ax.set_title(q['title'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …